# Figure Uncertainties underlying carbon

In [ ]:
# Libraries
import os
import xarray as xr
import rioxarray
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [ ]:
# Directories
dir_data = '../data/'
dir_nearth = '../data/naturalearth/'
dir01 = '../paper_deficit/output/01_prep/'
dir06 = '../paper_deficit/output/06_eval/'

---

In [ ]:
# Libraries
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask

# Initialize dask
cluster = SLURMCluster(
    queue='compute',                      # SLURM queue to use
    cores=24,                             # Number of CPU cores per job
    memory='256 GB',                      # Memory per job
    account='bm0891',                     # Account allocation
    interface="ib0",                      # Network interface for communication
    walltime='01:00:00',                  # Maximum runtime per job
    local_directory='../dask/',           # Directory for local storage
    job_extra_directives=[                # Additional SLURM directives for logging
        '-o ../dask/LOG_worker_%j.o',     # Output log
        '-e ../dask/LOG_worker_%j.e'      # Error log
    ]
)

# Scale dask cluster
cluster.scale(jobs=5)

# Configurate dashboard url
dask.config.config.get('distributed').get('dashboard').update(
    {'link': '{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'}
)

# Create client
client = Client(cluster)

client

In [ ]:
# Get coastline data
coastline110 = gpd.read_file(
    os.path.join(dir_data, 'naturalearth', 'ne_110m_coastline/ne_110m_coastline.shp'))

In [ ]:
# Get prepared data
ds_spawn_agbcunc = xr.open_zarr(
    os.path.join(dir01, 'ds_regridded_spawn_agbcunc.zarr'))
ds_spawn_bgbcunc = xr.open_zarr(
    os.path.join(dir01, 'ds_regridded_spawn_bgbcunc.zarr'))
ds_esabio_agbunc = xr.open_mfdataset(
    os.path.join(dir01, 'ds_regridded_esabio_agbsd*.zarr'), engine='zarr')
ds_huang_bgbunc = xr.open_zarr(
    os.path.join(dir01, 'ds_regridded_huang_bgbunc.zarr'))
ds_sgrids_socunc = xr.open_zarr(
    os.path.join(dir01, 'ds_regridded_soilgrids2020_ocsunc_0-30cm.zarr'))
ds_land = xr.open_zarr(
    os.path.join(dir01, 'ds_prep_copernicus_land_mask.zarr'))

# Create da_land
da_land = ds_land.copernicus_land_mask

In [ ]:
# Rename data variables of esabio data
for i in range(2015, 2022):
    ds_esabio_agbunc[i] = ds_esabio_agbunc[f'regridded_esabio_agbsd{i}'].rename(i)
    ds_esabio_agbunc = ds_esabio_agbunc.drop_vars(f'regridded_esabio_agbsd{i}')

In [ ]:
# Testing (faster, no transform)
fig, axs = plt.subplots(figsize=(15, 10), ncols=2, nrows=3)
axs = axs.ravel()

for i in range(0, 5):
    # coastline data
    coastline110.plot(ax=axs[i], 
                      color='#000000', linewidth=0.5)

def plot_map(da, ax, cbar_label):
    da.where(da_land == True) \
        .plot.imshow(ax=ax, robust=True, cmap='Reds',
                     cbar_kwargs=dict(label=cbar_label))

# Spawn agbc
plot_map(ds_spawn_agbcunc.regridded_spawn_agbcunc, axs[0], 'tC ha$^{-1}$')
plot_map(ds_esabio_agbunc.to_array('year').mean('year'), axs[1], 'Mg ha$^{-1}$')
plot_map(ds_spawn_bgbcunc.regridded_spawn_bgbcunc, axs[2], 'tC ha$^{-1}$')
plot_map(ds_huang_bgbunc.regridded_huang_bgbunc, axs[3], 'Mg ha$^{-1}$') # error data has much less datapoints than agb and bgb data, wrote email 26.08.2024
plot_map(ds_sgrids_socunc['regridded_soilgrids2020_ocsunc_0-30cm'], axs[4], 'tC ha$^{-1}$')

# Gsoc soc
# no uncertainty layer available, wrote e-mail 26.08.2024, waiting for answer 
axs[5].axis('off')

# Titles
axs[0].set_title('Spawn et al. 2020; AGBC 2010\nStandard error of prediction')
axs[1].set_title('Santoro & Cartus, 2024; Mean AGB 2015-2021\nStandard deviation')
axs[2].set_title('Spawn et al. 2020; BGBC 2010\nStandard error of prediction')
axs[3].set_title('Huang et al. 2021; BGB 2010\nStandard deviation')
axs[4].set_title('Poggio et al. 2021; SOC 0-30 cm\nRatio between inter-quantile range and median')
axs[5].set_title('FAO 2022 (GSOCmap v1.6.0)\nNo uncertainty estimates')
plt.tight_layout()

In [ ]:
# Plot
fig = plt.figure(figsize=(9, 9), dpi=600, constrained_layout=True)
fig.set_facecolor('#ffffff')

gs = fig.add_gridspec(3, 2,
                      hspace=0, wspace=0,
                      width_ratios=(.5, .5), 
                      height_ratios=(.33, .33, .33))

ax0 = fig.add_subplot(gs[0], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
ax1 = fig.add_subplot(gs[1], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
ax2 = fig.add_subplot(gs[2], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
ax3 = fig.add_subplot(gs[3], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
ax4 = fig.add_subplot(gs[4], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
ax5 = fig.add_subplot(gs[5], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
axs = [ax0, ax1, ax2, ax3, ax4, ax5]

for i in range(0, 6):
    coastline110.plot(ax=axs[i], transform=ccrs.PlateCarree(), 
                      color='#000000', linewidth=0.5)
    axs[i].set_extent((-180, 180, -60, 90), ccrs.PlateCarree())
    axs[i].axis('off')

def plot_map(da, ax, cbar_label):
    da.where(da_land == True) \
        .plot.imshow(ax=ax, transform=ccrs.PlateCarree(), 
                     robust=True, cmap='Reds',
                     cbar_kwargs=dict(label=cbar_label, shrink=0.6))

plot_map(ds_spawn_agbcunc.regridded_spawn_agbcunc, axs[0], 'tC ha$^{-1}$')
plot_map(ds_esabio_agbunc.to_array('year').mean('year'), axs[1], 'Mg ha$^{-1}$')
plot_map(ds_spawn_bgbcunc.regridded_spawn_bgbcunc, axs[2], 'tC ha$^{-1}$')
plot_map(ds_huang_bgbunc.regridded_huang_bgbunc, axs[3], 'Mg ha$^{-1}$') # error data has much less datapoints than agb and bgb data, wrote email 26.08.2024
plot_map(ds_sgrids_socunc['regridded_soilgrids2020_ocsunc_0-30cm']/10, axs[4], '')

# gsoc, soc
# no uncertainty layer available, wrote e-mail 26.08.2024, waiting for answer

# Titles
axs[0].set_title('Spawn et al. 2020 [S6]; AGBC 2010\nStandard error of prediction')
axs[1].set_title('Santoro & Cartus, 2024 [S7]; Mean AGB 2015-2021\nStandard deviation')
axs[2].set_title('Spawn et al. 2020 [S6]; BGBC 2010\nStandard error of prediction')
axs[3].set_title('Huang et al. 2021 [S8]; BGB 2010\nStandard deviation')
axs[4].set_title('Poggio et al. 2021 [S9]; SOC 0-30 cm\nRatio between inter-quantile range and median')
axs[5].set_title('FAO 2022 [S10]; SOC 0-30 cm\nNo uncertainty estimates')

# Export
plt.savefig(os.path.join(dir06, 'pdf/figs12_uncertainties_underlying_maps.pdf'), bbox_inches='tight', dpi=600)
plt.savefig(os.path.join(dir06, 'png/figs12_uncertainties_underlying_maps.png'), bbox_inches='tight', dpi=600);

In [ ]:
cluster.close()